In [2]:
import requests
import datetime
import pandas as pd
import yaml
import re
from bs4 import BeautifulSoup

In [3]:
r = requests.get(url='http://liquipedia.net/dota2/api.php?action=ask&query=[[%3A%2B]][[Has+teamid%3A%3A>0]]|%3FHas+location|%3FHas+region|%3FHas+id|%3FHas+name|sort%3Dhas+id|limit%3D500&format=json')
json = r.json()

In [4]:
lq = []
for lname, ldata in json.get('query').get('results').items():
    po = ldata.get('printouts')
    tnlow = re.sub(r'\W+', '', ldata.get('fulltext')).lower()
    tn = ldata.get('fulltext')
    tl = ldata.get('fullurl')
    try:
        rg = po.get('Has region')[0]
    except IndexError:
        rg = 'Nan' 
    try:
        loc = po.get('Has location')[0]
    except IndexError:
        loc = 'Nan'        
    lq.append(tnlow)

In [5]:
urldat = 'http://datdota.com/api/teams/performances?tier=1&tier=2&valve-event=does-not-matter&threshold=5&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&patch=6.87&patch=6.86&patch=6.85&patch=6.84&patch=6.83&patch=6.82&patch=6.81&patch=6.80&patch=6.79&patch=6.78&patch=6.77&patch=6.76&patch=6.75&patch=6.74&winner=either&after=01%2F01%2F2011&before=20%2F02%2F2018&duration=0%3B200&duration-value-from=0&duration-value-to=200'
htmldat = requests.get(urldat)
htmldat.encoding = 'utf-8'
il = yaml.load(htmldat.text)
il = il['data']

In [6]:
da = pd.DataFrame(columns=['name'])
dat = []

In [7]:
for l in il:
    name = re.sub(r'\W+', '', l['team']['name']).lower()
    da = da.append({'name': name}, ignore_index=True)
    dat.append(name)

In [8]:
eql = {}
eqd = {}
for l in lq:
    for d in dat:
        if d == l:
            eql[l] = d
            eqd[d] = l
            l = ''
            d = ''
        if ((d in l) or (l in d)) and (abs(len(l) - len(d)) < 5) and ((d != '') and (l != '')):
            eql[l] = d
            eqd[d] = l

In [9]:
#load to check which team missing in compare dictionary
df = pd.read_csv('data.csv', encoding = "ISO-8859-1")
df = df.dropna()

In [10]:
df.team = df.team.str.replace(r'\W+', '').str.lower()
tdf = df.team.unique().tolist()
tdf = sorted([re.sub(r'\W+', '', x).lower() for x in tdf])

In [11]:
for il in set(tdf).intersection(lq):
    tdf.remove(il)

In [12]:
tdf

['3dmax',
 'cisgame',
 'counterlogicgaming',
 'denialesports',
 'fnaticeu',
 'forlove',
 'ftdcluba',
 'lvgaming',
 'midasclubelite',
 'moscowfive',
 'quanticgaming',
 'sigmaint',
 'sneakynyxassassins',
 'teamdk',
 'teamunknown',
 'teamvgj',
 'teamzenith']

In [13]:
eql['3dmax'] = '3dmax'
eql['cisgame'] = 'cisgame'
eql['counterlogicgaming'] = 'counterlogicgaming'
eql['denialesports'] = 'denialesports'
eql['fnaticeu'] = 'fnatic'
eql['forlove'] = 'forlove'
eql['ftdcluba'] = 'ftdcluba'
eql['lvgaming'] = 'lvgaming'
eql['midasclubelite'] = 'midasclub'
eql['moscowfive'] = 'moscowfive'
eql['quanticgaming'] = 'quanticgaming'
eql['sigmaint'] = 'sigmaint'
eql['sneakynyxassassins'] = 'sneakynyxassassins'
eql['teamdk'] = 'teamdk'
eql['teamvgj'] = 'teamvgj'
eql['teamzenith'] = 'zenith'
del eql['fnaticeu']

In [23]:
def save_obj(obj, name ):
    import pickle
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [26]:
save_obj(eql, 'lqtodat')

In [14]:
cp = pd.DataFrame.from_dict(list(eql.items()))

In [16]:
cp = cp.rename(columns={0:'lq', 1:'dat'})
cp.head()

,lq,dat
0,monkeybusiness,monkeybusiness
1,,
2,4friendschrillee,4friendschrillee
3,4protectfive,4protectfive
4,5jungz,5jungz


In [17]:
cp.to_csv('lqtodat.csv')